In [ ]:
# default_exp core

In [ ]:
#export
from urllib.request import urlopen
from bs4 import BeautifulSoup
import statistics as st

# intel_invest

> A package to obtain the relevant figures for a company and apply some rules of thumb precribed in the book Intelligent Investor, by Benjamin Graham.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
type(float("NaN"))

float

In [ ]:
#export 
def _flotify(n: str) -> float:
    try:
        return float(n)
    except ValueError:
        return float("NaN")

This is the main class which contains within it all the functionality for decision making.

In [ ]:
# export
class Stock:
    def __init__(self, symbol: str) -> None:
        self.symbol = symbol
        self.url_financial = (
            f"http://www.marketwatch.com/investing/stock/{symbol}/financials"
        )
        self.url_profile = (
            f"http://www.marketwatch.com/investing/stock/{symbol}/profile"
        )
        self.soup_financial = self._get_soup(self.url_financial)
        self.soup_profile = self._get_soup(self.url_profile)

    def _get_soup(self, url: str) -> str:
        return BeautifulSoup(urlopen(url).read(), features="html.parser")

    def get_price(self) -> float:
        """Fetchs the price of stock."""
        price = self.soup_financial.find("bg-quote", {"class": "value"})
        return _flotify(price.text)

    def get_esp(self) -> list:
        """Fetches earning per share or last 5 years."""
        titles = self.soup_financial.findAll(
            "td", {"class": "overflow__cell fixed--column"}
        )
        esp_title = [t for t in titles if "EPS (Basic)" in t.text]
        if esp_title:
            esp_title = esp_title[0]
        eps = [
            _flotify(t.text)
            for t in esp_title.findNextSiblings(attrs={"class": "overflow__cell"})[:5]
        ]
        return eps

    def max_price(self) -> float:
        """Calculates the maximum price one should pay for a stock.

        Max stock price = 25 * 5 year average EPS.
        """
        return 25 * st.mean(self.get_esp())

    def get_pe(self) -> float:
        """Fetches price to earning ratio (current)."""
        label_val = zip(
            self.soup_profile.findAll("td", {"class": "table__cell w75"}),
            self.soup_profile.findAll("td", {"class": "table__cell w25"}),
        )
        pe = [val.text for lab,val in label_val if "P/E Current" in lab.text]
        return _flotify(pe[0])
    
    def get_pbv(self) -> float:
        """Fetches price to book value."""
        label_val = zip(
            self.soup_profile.findAll("td", {"class": "table__cell w75"}),
            self.soup_profile.findAll("td", {"class": "table__cell w25"}),
        )
        pbv = [val.text for lab,val in label_val if "Price to Book Ratio" in lab.text]
        return _flotify(pbv[0])
    
    def will_ben_buy_2(self):
        """Ben Graham's rule 2 is that P/E * P/BV < 22.5."""
        return self.get_pe() * self.get_pbv() < 22.5
    
    def will_ben_buy_1(self):
        """Ben Graham's rule 1 is that P < 5 yr average of EPS."""
        return self.get_price() < self.max_price()

In [ ]:
s = Stock("ABT")

In [ ]:
s.will_ben_buy_1()

False

In [ ]:
s.will_ben_buy_2()

False

In [ ]:
s.max_price()

35.65

In [ ]:
s.get_pe()

47.13

In [ ]:
s.get_price()

117.2

In [ ]:
s.get_pbv()

5.92